In [1]:
from scraper import *
from downloader import *
from analytics import *

# 1. 数据获取
## 1.1 机构

In [ ]:
%%time
df_companies = Company(headless=False).get_company_list(save_to_local=False)

## 1.2 人员列表
所有公司人员列表运行时间

In [3]:
%%time
# 单个公司人员列表：不含保存到本地的功能
df_people = People(headless=True).get_people_list(company_ID='1999152')


100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


中德证券有限责任公司：表格共有11列，242行。


,PPP_ID,姓名,性别,学历,证书编号,执业机构,执业岗位,证书取得日期,证书有效截止日期,注册变更记录,诚信记录
0,76F3A6297BD79045E053D651A8C01A5D,韩正奎,男,硕士研究生,S1520712100001,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,5条,无
1,76F3A62973929045E053D651A8C01A5D,崔学良,男,硕士研究生,S1520712100002,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,3条,无
2,76F3A62973939045E053D651A8C01A5D,单晓蔚,女,本科,S1520712100003,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,4条,无
3,76F3A6296FCC9045E053D651A8C01A5D,牛岗,男,硕士研究生,S1520712100005,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,3条,无
4,76F3A6297FB19045E053D651A8C01A5D,李铁,男,博士研究生,S1520109070003,中德证券有限责任公司,一般证券业务,2009-07-22,2018-12-31,4条,无
5,76F3A6298B0B9045E053D651A8C01A5D,万新,女,硕士研究生,S1520110020001,中德证券有限责任公司,一般证券业务,2010-02-03,2018-12-31,2条,无
6,76F3A62980F69045E053D651A8C01A5D,顾蓓菁,女,本科,S1520109110103,中德证券有限责任公司,一般证券业务,2009-11-12,2019-12-31,3条,无
7,76F3A62987F29045E053D651A8C01A5D,毛传武,男,本科,S1520712100009,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,4条,无
8,76F3A6297DFE9045E053D651A8C01A5D,梁炜,男,硕士研究生,S1520712100007,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,5条,无
9,76F3A6298EB69045E053D651A8C01A5D,陈亚东,男,硕士研究生,S1520712100010,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,4条,无


CPU times: user 296 ms, sys: 50.1 ms, total: 346 ms
Wall time: 10.5 s


In [2]:
%%time
# 所有公司人员列表：默认自动保存到本地
## 去除：单个公司的df_people的display和tqdm的每页提示；
## 保留：总的tqdm和执行到当期的公司名称和页数提示
## 提速：降低每页sleep到1.5秒
## 稳定性：每爬完一个公司都保存，保证断点续传；放到pickle？
df_people_full = People(headless=False).get_people_list_full(save_to_local=False, sleep_time=1)

表格共有12列，121行。


,序号,机构名称,从业人员数,一般证券业务,证券投资咨询业务(其他),证券经纪业务营销,证券经纪人,证券投资咨询业务(分析师),证券投资咨询业务(投资顾问),保荐代表人,投资主办人,机构ID
0,1,爱建证券有限责任公司,1324,707,0,3,538,7,52,7,10,1999082
1,2,安信证券股份有限公司,8874,6037,0,190,1584,74,881,97,11,1999145
2,3,北京高华证券有限责任公司,135,98,0,0,0,16,21,0,0,1999142
3,4,渤海汇金证券资产管理有限公司,147,122,0,0,0,0,0,0,25,609
4,5,渤海证券股份有限公司,1788,1260,0,33,325,24,134,12,0,1999115
5,6,财达证券股份有限公司,2347,1658,0,14,467,22,161,6,19,1999040
6,7,财富证券有限责任公司,2731,1380,0,25,966,15,321,7,17,1999053
7,8,财通证券股份有限公司,3582,2274,0,2,839,35,393,39,0,1999016
8,9,长城国瑞证券有限公司,742,556,0,3,124,4,28,11,16,1999020
9,10,长城证券股份有限公司,4061,2280,0,1,1310,31,344,57,38,1999107


 25%|██▌       | 18/72 [00:20<01:00,  1.12s/it]

爱建证券有限责任公司：表格共有11列，786行。


WebDriverException: Message: chrome not reachable
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.13.6 x86_64)


In [ ]:
df_people_full.to_json('列表_从业人员.json',orient='table')

In [ ]:
# 二次读取数据
df_people_full = pd.read_csv('列表_从业人员.csv', encoding='gb18030')#.iloc[0,6]

## 1.3 个人信息
只涉及
1. 注册变更记录
2. 照片路径

In [ ]:
%%time
# 保存单个人的照片
People().get_person_info_img('76B6EA9DB8E33C0DE053D651A8C06CD1')

# 2. 分析：基础
## 2.1 性别比率

In [ ]:
json_gender = Basic().basic_gender()

In [ ]:
json_gender[2]

In [ ]:
def basic_date_registration(df=df_people_full):
    df_date_registration = df.groupby(pd.Grouper(key='证书取得日期')).size().to_frame('人数').sort_values(by='证书取得日期')
    return df_date_registration

# 3. 分析：高级
## 3.1 高分妹发掘器

In [ ]:
df_candidates = df_people_full[df_people_full['性别'] == '女'].sort_values(by=['证书取得日期'])
# 获得照片路径

# 颜值判定：使用腾讯人脸识别API判定魅力

# 下载高颜值的女生照片并重命名

In [ ]:
df_candidates